In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
# wavenumber and wavelength
k = 2.0 
lamb =  2.0 * np.pi / k

# domain [la,lb]
la = -8.0 * lamb
lb = 8.0 * lamb

# define the mesh for observation
N_obs = 240
h_obs = (lb - la) / N_obs
mesh_obs = np.linspace(la, lb, (N_obs + 1))

mesh_mid_obs = np.zeros(N_obs + 1)

mesh_mid_obs[0] = la

for od in range(N_obs):
    mesh_mid_obs[od + 1] = (mesh_obs[od] + mesh_obs[od+1])/2.0

# transform numpy mesh to torch mesh
x_mid_obs = torch.tensor(mesh_mid_obs, dtype = torch.float32, requires_grad = True)
x_mesh_obs = torch.tensor(mesh_obs, dtype = torch.float32, requires_grad = True)

# define the maximum number of testing points
N_inv = 480

# define the max and min of surface height
hmax = 0.2 * lamb
hmin = -0.2 * lamb

In [3]:
# angle of grazing
alpha = torch.tensor([[-2.0 * np.pi / 3.0]])

# the incident wave, which is a plane wave
def inc_field(x, z):
    g = torch.cos(alpha) * x + torch.sin(alpha) * z
    return torch.exp(1.0 * 1j * k * g)

# observation height 
zp = lamb * 0.4

# Hankel function of zeroth order of first kind
def hankel(sth):
        return torch.special.bessel_j0(sth) + 1j * torch.special.bessel_y0(sth)
    
# method of moments
# note here this function produces the phaseless total field
def MOM(x_mid, hei_vec, heid_vec):
    ''' This function implements the method of moments.
     x_mid is the mesh points.
     hei_vec is the torch tensor for the surface height.
     heid_vec is the torch tensor of the derivative of surface height.
     it reaturns the torch tensor of phaseless total field.'''
    
    h = x_mid[100] - x_mid[99]
    N = x_mid.numel() - 1
    
    # aseemble surface incident field
    inc_vec =  torch.zeros(N, dtype=torch.cfloat)
    for i in range(N):
        inc_vec[i] = inc_field(x_mid[i+1], hei_vec[i+1])
    
    # aseemble matrix A
    A =  torch.zeros((N, N), dtype=torch.cfloat)
    
    # off-diagonal terms
    def getA_offdiag1(nn):
        ll = torch.arange(1, nn)
        ptl = 1.0 * k * torch.sqrt((x_mid[nn] - x_mid[ll]) ** 2 + (hei_vec[nn] - hei_vec[ll])**2)
        sd = torch.sqrt(1.0 + heid_vec[ll]**2)
        return 1.0 * h * 1j / 4.0 * hankel(ptl) * sd 
    def getA_offdiag2(nn):
        ll = torch.arange(nn + 1, N + 1)
        ptl = 1.0 * k * torch.sqrt((x_mid[nn] - x_mid[ll]) ** 2 + (hei_vec[nn] - hei_vec[ll])**2)
        sd = torch.sqrt(1.0 + heid_vec[ll]**2)
        return 1.0 * h * 1j / 4.0 * hankel(ptl) * sd 
    
    # diagonal term
    def getA_diag(nn):
        gamma = 0.5772156649
        term1 = 1.0 * 1j / 4.0 * h * (1.0 + 2.0 * 1j / torch.pi * gamma) 
        sd = torch.sqrt(1.0 + heid_vec[nn]**2)
        term2 = - (1.0 * h) / (2.0 * torch.pi) * (torch.log(k * sd * h / 4.0) - 1.0)
        return (term1 + term2) * sd
    
    # assemble matrix A
    for nu in range(N):
        A[nu, 0:nu] = getA_offdiag1(nu + 1)
        A[nu, nu+1:N] = getA_offdiag2(nu + 1)
        A[nu, nu] = getA_diag(nu + 1)
    
    # solve for surface current
    phid = torch.linalg.solve(A, inc_vec)
    
    zz = torch.tensor([zp])
    
    # assemble matrix B
    B =  torch.zeros((N, N), dtype=torch.cfloat)
    
    # components of matrix B
    def getB(nn):
        rr = torch.arange(1, N + 1) 
        ptl = 1.0 * k * torch.sqrt((x_mid[nn] - x_mid[rr]) ** 2 + (zz - hei_vec[rr])**2)
        sd = torch.sqrt(1.0 + heid_vec[rr]**2)
        return - 1.0 * h * 1j / 4.0 * hankel(ptl) * sd 
    
    for nd in range(N):
        B[nd, 0:N] = getB(nd + 1)

    # calculate scattered field
    phis = torch.matmul(B, phid)
    
    # assemble incident wave along measurement line
    inc_vec_zz = torch.zeros(N, dtype=torch.cfloat)
    for nt in range(N):
        inc_vec_zz[nt] = inc_field(x_mid[nt+1], zz)
    
    # return the amplitude of total field
    return torch.abs(phis + inc_vec_zz)

In [4]:
# load the data of actual surface 
hei_actual_torch = torch.load('./vector_true_surface.pt')

# load the data list of the interpolated scattered field
# data without noise
data_total_field_without_noise = torch.load('./data_phaseless_total_field_without_noise.pt')
# data with noise
data_total_field_with_noise = torch.load('data_phaseless_total_field_with_noise.pt')

In [5]:
# main structure for the neural network

class Net(nn.Module):
    # NL: the number of hidden layers
    # NN: the number of vertices in each layer
    def __init__(self, NL, NN):
        super(Net, self).__init__()

        self.input_layer = nn.Linear(1, NN)

        self.hidden_layers = nn.ModuleList([nn.Linear(NN, NN) for i in range(NL)])

        self.output_layer = nn.Linear(NN, 1)
        
    def forward(self, x):
        o = self.act(self.input_layer(x))

        for i, li in enumerate(self.hidden_layers):
            o = self.act(li(o))
        
        out = self.output_layer(o)
        
        return out

    def act(self, x):
        return x * torch.sigmoid(x)

In [6]:
# at each epoch, we generate a mesh of size N_t, N_t is inside [N_obs, N_inv]
def generate_mesh():
    
    # first generate a random number N_t
    N_t = torch.randint(N_obs, N_inv + 1, (1,)).item()
    
    # identify the position in the data list
    idx = N_t - N_obs
    
    # generate the mesh
    x_mesh = torch.linspace(la,lb, N_t+1, requires_grad = True).view(-1,1)
    x_train = torch.zeros(N_t + 1)
    x_train[0] = la
    for od in range(N_t):
        x_train[od + 1] = (x_mesh[od] + x_mesh[od+1])/2.0
        
    return idx, x_train, N_t

# normalization of a tensor
def normalize_tensor(input_tensor):
    
    min_val = input_tensor.min()
    max_val = input_tensor.max()
    normalized_tensor = 2 * (input_tensor - min_val) / (max_val - min_val) - 1
    
    return normalized_tensor

# the inverse normalization of a tensor
def inverse_normalize_tensor(normalized_tensor, original_max, original_min):
    
    # Compute the scale and shift parameters
    scale = (original_max - original_min) / 2
    shift = (original_max + original_min) / 2
    
    inverse_normalized_tensor = normalized_tensor * scale + shift
    return inverse_normalized_tensor

# the automatic differentiation gives the derivative of normalization of output
# w.r.t. normalizaton of input, applying the chain rule function leads to the
# actual derivative of target output w.r.t. target input
def chain_rule(derivative_tensor, output_max, output_min, input_max, input_min):
    return (output_max - output_min) * derivative_tensor / (input_max - input_min)

In [7]:
# set up parameters for PINN neural networks
# NL is number of hidden layers and NN is the number of neurons per hidden layer
net = Net(NL = 6, NN = 512)
# choose an optimizer and learning rate
optimizer = optim.Adam(net.parameters(), lr = 1E-3)
# set number of iterations
N_PINN = 3000

# this is the main loop for the PINN
for t in range(N_PINN):
    
    # first step, generate N_t and the mesh
    idx, x_train_ori, N_t = generate_mesh()
    
    # normalization of points
    x_train = normalize_tensor(x_train_ori)
    
    # forward propagation
    x_train = x_train.view(-1,1)
    y = net(x_train)
    
    # use automatic differentiation to find derivatives
    dy = torch.autograd.grad(y, x_train, grad_outputs= torch.ones(y.size()), create_graph = True)[0] 
    
    # transform the surface to 1D vector and apply inverse normalization
    hei_pred_net = y.view(-1)
    hei_pred = inverse_normalize_tensor(hei_pred_net, hmax, hmin)
    
    # transform derivative to 1D vector and apply the chain rule
    heid_pred_net = dy.view(-1)
    heid_pred = chain_rule(heid_pred_net, hmax, hmin, x_train_ori.max(), x_train_ori.min())

    # use MOM to calculate the total field
    field_pred = MOM(x_train_ori, hei_pred, heid_pred)
    
    # identify the corresponding observed scattered data
    field_obs = data_total_field_with_noise[idx]

    # MSE loss of the residual of equation
    loss_eqn = nn.MSELoss()(field_pred, field_obs)
    
    # find the predicted surface values on the close-boundary points
    indices_bc = torch.tensor([0, 1, 2, 3, 4, N_t, N_t - 1, N_t - 2, N_t - 3, N_t - 4])
    pred_bc = hei_pred[indices_bc]
    
    # MSE loss of boundary values
    loss_bc = nn.MSELoss()(pred_bc, torch.zeros(10))
    
    # the total loss is the combination 
    loss = loss_eqn + loss_bc 
    
    # optimizing step with backward propagation
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    
    # print(f"Epoch {t}, Loss: {loss.item()}")
    if (t + 1) % 100 == 0:
        print(f"Epoch {t + 1}, Loss: {loss.item()}")

Epoch 100, Loss: 0.06607130169868469
Epoch 200, Loss: 0.06279180943965912
Epoch 300, Loss: 0.04489124193787575
Epoch 400, Loss: 0.03558586910367012
Epoch 500, Loss: 0.036156922578811646
Epoch 600, Loss: 0.3710682988166809
Epoch 700, Loss: 0.039253316819667816
Epoch 800, Loss: 0.03342565894126892
Epoch 900, Loss: 0.01957095041871071
Epoch 1000, Loss: 0.021058855578303337
Epoch 1100, Loss: 0.008052891120314598
Epoch 1200, Loss: 0.0010776676936075091
Epoch 1300, Loss: 0.0004969916772097349
Epoch 1400, Loss: 0.001431651646271348
Epoch 1500, Loss: 0.00028632156318053603
Epoch 1600, Loss: 0.0004582072142511606
Epoch 1700, Loss: 0.0005113692604936659
Epoch 1800, Loss: 0.0006212764419615269
Epoch 1900, Loss: 0.0016915624728426337
Epoch 2000, Loss: 0.0013159916270524263
Epoch 2100, Loss: 0.0008687266381457448
Epoch 2200, Loss: 0.0002966840111184865
Epoch 2300, Loss: 0.00037571435677818954
Epoch 2400, Loss: 0.000618560123257339
Epoch 2500, Loss: 0.00037275574868544936
Epoch 2600, Loss: 0.0009233

In [ ]:
# normalization of testing points
x_mesh_test = normalize_tensor(x_mesh_obs)

# forward propagation and inverse normalization to obtain the result surface height
y = net(x_mesh_test.view(-1,1)).view(-1)
hei_pred_output = inverse_normalize_tensor(y, hmax, hmin)

# transform torch tensors to numpy arrays
hei_actual_vec = hei_actual_torch.detach().numpy()
hei_pred_vec = hei_pred_output.detach().numpy()

# plot the result
plt.figure(figsize=[8, 6])
line1, = plt.plot(mesh_mid_obs / lamb, hei_actual_vec / lamb, color='r', linewidth = 2)
line2, = plt.plot(mesh_mid_obs / lamb, hei_pred_vec / lamb, color='b', linewidth=2, linestyle = '--', fillstyle = 'none')
plt.xlabel(r'$x/{\lambda}$', fontsize = 15)
plt.ylabel(r'$z/{\lambda}$', fontsize = 15)
leg=plt.legend([line1, line2],
        ['actual surface',
        'reconstructed surface'],
        prop={'size': 15}, loc = "best")
plt.gca().tick_params(labelsize=15)
plt.show()

In [ ]:
# calculate the l2 norm error
error_vec = hei_actual_vec - hei_pred_vec
print("l2 norm error is: ", np.linalg.norm(error_vec) / np.linalg.norm(hei_actual_vec))